In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Clone github repository setup
# import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'My Drive/Colab Notebooks/Twump'
# replace with your Github username 
GIT_USERNAME = "sami12rom" 
# definitely replace with your
GIT_TOKEN = "feb1339ef41c3563c071ed36b88da8198847d189"  
# Replace with your github repository in this case we want 
# to clone deep-learning-v2-pytorch repository
GIT_REPOSITORY = "TwitterNLP" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"    

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive/My Drive/Colab Notebooks/Twump
mkdir: cannot create directory ‘/content/drive/My Drive/Colab Notebooks/Twump’: File exists
GIT_PATH:  https://feb1339ef41c3563c071ed36b88da8198847d189@github.com/sami12rom/TwitterNLP.git


In [3]:
%cd "{PROJECT_PATH}"
!git clone "{GIT_PATH}" #clone the github repository



/content/drive/My Drive/Colab Notebooks/Twump
Cloning into 'TwitterNLP'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [7]:
%cd "{PROJECT_PATH}/TwitterNLP"
!git branch

/content/drive/My Drive/Colab Notebooks/Twump/TwitterNLP
* master


In [0]:
#!pip install nltk

In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [0]:
df.sort_values(by='date', ascending=False)

In [0]:
oldtweets = df[['id','date','content','retweets','favorites']].sort_values(by='date', ascending=False)
oldtweets.head()

In [0]:
def standardize_text(df, text_field):
    # normalize by turning all letters into lowercase
    df[text_field] = df[text_field].str.lower()
    return df


# call the text cleaning function
cleantext = standardize_text(oldtweets, "content")


In [0]:
## [PREPROCESS] Tokenize

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

cleantext["tokens"] = cleantext["content"].apply(tokenizer.tokenize)
cleantext.head()

In [0]:
#Lemmatizing
wn=nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
  text = [wn.lemmatize(word) for word in tokenized_text]
  return text
  

#Stemming
ps = nltk.PorterStemmer()
def stemming(tokenized_text):
  text = [ps.stem(word) for word in tokenized_text]
  return text

table = cleantext
#table['processed'] = table['tokens'].apply(lambda x: stemming(x))
table['processed'] = table['tokens'].apply(lambda x: lemmatizing(x))
table.head()

In [0]:
all_words = [word for tokens in cleantext["processed"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in cleantext["processed"]]

VOCAB = sorted(list(set(all_words)))

print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))

In [0]:
print (VOCAB[0:100])

# What are the most commonly occuring words
from collections import Counter
count_all_words = Counter(all_words)

# get the top 100 most common occuring words
count_all_words.most_common(100)

In [0]:
unwantedwords='!@#"', 'realDonaldTrump','realdonaldtrump', 'Donald', '.', '&', '-', 'A', '"', 'the', '','com', 'http'
clean_words=all_words
for char in unwantedwords:
    clean_words = [w.replace(char,"") for w in clean_words]

count_all_words = Counter(clean_words)

# get the top 100 most common occuring words
count_all_words.most_common(100)

In [0]:
from nltk.corpus import stopwords
sr= stopwords.words('english')
clean_tokens = clean_words

clean_tokens = [token for token in clean_tokens if token not in sr]
clean_tokens = [token for token in clean_tokens if token]
#for token in clean_words:
    #if token in sr:
        #clean_tokens.remove(token)
clean_tokens

![](https://drive.google.com/uc?id=1Z2Sr70L-Dj_JNGHlnV-WLGlfPigt-8pW)

In [0]:
tagged = nltk.pos_tag(clean_tokens)
tagged[0:10]

In [0]:
freq = nltk.FreqDist(clean_tokens)
freqtable = list(freq.items())
print(type(freqtable))
freqtable = pd.DataFrame(data=freqtable, columns=['word','freq'])
freqtable['tag'] = (nltk.pos_tag(freqtable['word']))
freqtable['tag'] = [str(i) for i in freqtable['tag'].str[1]]
freqtable = freqtable.sort_values(by='freq',ascending=False)
print(freqtable.head())
#freqtable = [str(key) + ':' + str(val) for key,val in freq.items()]
#freqtable = pd.DataFrame(data=freqtable, columns=['word'])
#freqtable['freq']=freqtable['word'].str.split(':').str[1]
#freqtable['word']=freqtable['word'].str.split(':').str[0]
#freqtable.head()
#print(freqtable[0:6])
#for key,val in freq.items():
    #print(str(key) + ':' + str(val))
freq.plot(20, cumulative=False)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer='char_wb')
X_counts = count_vect.fit_transform(freqtable['word'])
print(X_counts.shape)
print(count_vect.get_feature_names())